Me resultó interesante un artículo publicado en la web ResearchGate sobre una librería para validación de datos dentro de un DataFrame de Pandas. <br>
Su nombre es Pandera, y provee una API flexible, expresiva e intuitiva para la validación de datos utilizando reglas y criterios predefinidos. Diseñada en un principio para facilitar la validación de Dataframes de pandas en tiempo de ejecución, aunque con el tiempo se fue extendiendo para soportar estructuras de Dask, Modin y Spark.

A modo de introducción, el autor, Niels Bantilan, argumenta que los Dataframes pueden convertirse en estructuras muy complejas luego de sufrir varias transformaciones, lo que puede dificultar pensarlos y analizarlos a partir de sus propiedades y contenidos. Por ello, el proceso de validación de datos, puede llevar mucho trabajo cognitivo y de desarrollo. <br>
Pandera se enfoca en hacer este proceso de validación fácil, intuitivo, flexible y expresivo.

El autor define la validación de datos como el proceso necesario para verificar si un conjunto de datos cumple o no con criterios de aceptación determinados. Esto incluye ciertas reglas definidas como puede ser los tipos de datos permitidos o rangos de valores válidos. <br> 
La validación se puede entender como una función cuya entrada son los datos a validar, y la salida es un valor booleano, tal que: validation(data) -> {True, False}

Asimismo, se hace una distinción entre los diferentes tipos de reglas validación, como por ejemplo:
- Técnicas: Se validan los tipos de datos, la estructura de los mismos, si pueden o no ser únicos o nulos, de forma más genérica.
- De dominio: Se validan los datos en sí para el contexto específico de trabajo/estudio. Por ejemplo rangos de valores permitidos, algún string dentro de un set predefinido, etc.
- Determinísticas vs Probabilísticas: Las primeras usan reglas basadas en reglas lógicas o dependencias funcionales, sin ninguna aleatoriedad. Mientras que las probabilísticas incorporan incertidumbre y aleatoriedad, como por ejemplo, pruebas de hipótesis.

Se hace hincapié en que la validación de datos siempre es necesaria, ya que es prácticamente imposible que tanto el código como los datos a procesar sean perfectos y sin errores, por lo tanto la validación es una parte importante en la cadena de procesos a realizar sobre un dataset previo a su análisis, procesamiento y visualización. De lo contrario, muy probablemente tendremos inferencias y visualizaciones incorrectas, así como comportamientos inesperados en nuestros modelos.<br>
La validación se realiza iterativamente junto con el análisis exploratorio, añadiendo y modificandolas hasta llegar al resultado esperado, y sirve asimismo a modo de documentación sobre cuál debe ser el formato válido para el dataset entrante.

Se mencionan los principios de diseño tenidos en cuenta al desarrollar el proyecto pandera, ellos son:
- Familiaridad para usuarios de pandas.
- Compatibilidad con flujos de trabajo diversos.
- Facilidad para definir reglas personalizadas.
- Debugging sencillo gracias a su interface.
- Fluida integración con el código existente.


Respecto a su arquitectura, el autor menciona que en pandera existen los esquemas que actúan como contratos que los DataFrames deben cumplir. Estos contratos especifican propiedades determinísticas y estadísticas que deben cumplirse para considerar como válido al dataframe. Entonces, luego de un procesamiento inicial de datos, se llega al "schema validator", donde si los datos son válidos, se devuelven, de lo contrario se lanza una excepción de tipo SchemaError con detalles de los errores y la data inválida en un nuevo dataframe de pandas.

Por otro lado, en el artículo de la web Medium, donde se hace referencia al artículo original de Niels Bantilan, se destacan algunos de los escenarios donde pandera puede ser de utilidad, destacando que no encontraremos grandes y complejos algoritmos en la librería, pero que funciona de buena manera para su objetivo de validación de datos, y siendo de muy fácil aprendizaje y usabilidad. 
En el caso de necesitar rápidamente un MVP (producto mínimo viable) donde se manejen grandes cantidades de consultas SQL para obtener datos, pandera puede servir como una buena base para validar la calidad necesaria de los datos. 
También en algunas startups, por ejemplo, donde es común enfrentarse a numerosos problemas de calidad de datos debido a la falta de pruebas unitarias, cambios frecuentes en el producto y ausencia de documentación, pandera puede ayudar a resolver problemas en los datos de manera eficiente y rápida.

In [2]:
# Se necesita instalar el modulo pandera ya que no viene incluido en jupyter notebook
%pip install pandera

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 2.1 MB/s eta 0:00:000:00:01m eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [163]:
# Función para descargar el csv a utilizar en la prueba de concepto
import urllib

def download_file(url, dest):
    urllib.request.urlretrieve(url, dest)
    return
    
students_url = "https://www.plus2net.com/python/download/student.csv"
students_file_name = "students.csv"

download_file(students_url, students_file_name)
print(f'Archivo {students_file_name} descargado correctamente.')



Archivo students.csv descargado correctamente.


In [169]:
# Cargo el csv descargado utilizando la función read_csv() de pandas.
students = pd.read_csv("students.csv", sep=',')
display(students)


,id,name,class,mark,gender
0,1,John Deo,Four,75,female
1,2,Max Ruin,Three,85,male
2,3,Arnold,Three,55,male
3,4,Krish Star,Four,60,female
4,5,John Mike,Four,60,female
5,6,Alex John,Four,55,male
6,7,My John Rob,Fifth,78,male
7,8,Asruid,Five,85,male
8,9,Tes Qry,Six,78,male
9,10,Big John,Four,55,female


Los Schemas en pandera son objetos inicializados con reglas de validación. Si los datos son válidos, la función validate devolverá el mismo set de datos. Caso contrario, se lanza un SchemaError.

Defino mi Schema para validar el DataFrame. Como reglas básicas para validar la lista de estudiantes, identifico los tipos de datos de cada columna.
Como quiero que falle, a la columna id le indico que debería ser string, cuando en realidad es integer.
Como resultado, se lanza un SchemaError, indicando que la serie "id" no es string sino integer, y mostrando las filas donde esa validación no se cumple, todas en este caso.

In [170]:
# Validación simple sobre los tipos de dato
import pandera as pa
from pandera import Column, DataFrameSchema

schema = pa.DataFrameSchema(
    {
        "id": Column(str),
        "name": Column(str),
        "class": Column(str),
        "mark": Column(int),
        "gender": Column(int)
    }
)

try:
    # Va a fallar porque la columna "id" no es string, sino integer
    schema.validate(students)
except pa.errors.SchemaError as e:
    print(f'Pandera SchemaError: {str(e)}')

Pandera SchemaError: expected series 'id' to have type str:
failure cases:
    index  failure_case
0       0             1
1       1             2
2       2             3
3       3             4
4       4             5
5       5             6
6       6             7
7       7             8
8       8             9
9       9            10
10     10            11
11     11            12
12     12            13
13     13            14
14     14            15
15     15            16
16     16            17
17     17            18
18     18            19
19     19            20
20     20            21
21     21            22
22     22            23
23     23            24
24     24            25
25     25            26
26     26            27
27     27            28
28     28            29
29     29            30
30     30            31
31     31            32
32     32            33
33     33            34
34     34            35


En este siguiente ejemplo corrijo los tipos de datos, pero agrego 2 nuevas filas al DataFrame, pasándoles None en la columna "gender". 
Esto hará fallar la validación nuevamente, en este caso por existir valores nulos en esa columna.

In [181]:
# Validación simple de las columnas y sus tipos de datos
schema = pa.DataFrameSchema(
    {
        "id": Column(int),
        "name": Column(str),
        "class": Column(str),
        "mark": Column(int),
        "gender": Column(str)
    }
)

try:
    students.loc[35] = [36, 'Diego', 'Five', 89, None]
    students.loc[36] = [37, 'Rocio', 'Nine', 95, None] 
    # Va a fallar porque la columna "gender" tiene valores None (nulos)
    schema.validate(students)
except pa.errors.SchemaError as e:
    # Muestra un Dataframe con el index y el valor que rompió el Schema
    print(f'Pandera SchemaError: {str(e)}')

Pandera SchemaError: non-nullable series 'gender' contains null values:
35    None
36    None
Name: gender, dtype: object


Puedo especificar que determinada columna acepte nulos. En este ejemplo la validación funcionará.

In [175]:
# Validación aceptando nulls en gender
schema = pa.DataFrameSchema(
    {
        "id": Column(pa.Int),
        "name": Column(str),
        "class": Column(str),
        "mark": Column(pa.Int),
        "gender": Column(str, nullable=True)
    }
)

try:
    # Se aceptan los valores None en "gender"
    schema.validate(students)
    print("Data Valida")
except pa.errors.SchemaError as e:
    # Muestra un Dataframe con el index y el valor que rompió el Schema
    print(f'Pandera SchemaError: {str(e)}')

Data Valida


Otro de los atributos que puedo especificar en el objeto Column es unique, en este caso, estoy pidiendo que la columna "class" tenga valores únicos. Como eso no sucede, la validación vuelve a fallar, ya que dicha columna posee valores duplicados.

In [177]:
# Validación sin permitir duplicados en class
schema = pa.DataFrameSchema(
    {
        "id": Column(int),
        "name": Column(str),
        "class": Column(str, unique=True),
        "mark": Column(int),
        "gender": Column(str, nullable=True)
    }
)

try:
    # No se permiten valores duplicados en la columna "class"
    schema.validate(students)
    print("Data Valida")
except pa.errors.SchemaError as e:
    # Muestra un Dataframe con el index y el valor que rompió el Schema
    print(f'Pandera SchemaError: {str(e)}')

Pandera SchemaError: series 'class' contains duplicate values:
0      Four
1     Three
2     Three
3      Four
4      Four
5      Four
7      Five
8       Six
9      Four
10      Six
11      Six
12    Seven
13    Seven
14     Four
15     Four
16      Six
17     Five
18     Nine
19     Nine
20     Four
21    Seven
23    Seven
24    Seven
25    Seven
26    Three
27    Seven
28    Seven
29      Six
30     Four
31    Seven
32      Six
33    Seven
34      Six
35     Five
36     Nine
Name: class, dtype: object


Los objetos centrales en pandera son DataFrameSchema, Column y Check. Con ellos se pueden definir las reglas de validación requeridas por el schema para operar y validar los DataFrames.
En este caso, agregando un Check dentro de Column, estoy pidiendo que los valores de la columna "id" estén dentro del rango [0,9], lo cual no se cumple.

In [178]:
# Validación para que los ids sean valores entre 0 y 9
schema = pa.DataFrameSchema(
    {
        "id": Column(int, Check.isin(range(10))),
        "name": Column(str),
        "class": Column(str, unique=False),
        "mark": Column(int),
        "gender": Column(str, nullable=True)
    }
)

try:
    # Los ids van de 1 a 36 en este dataframe, por lo cual no se cumple la validación
    schema.validate(students)
    print("Data Valida")
except pa.errors.SchemaError as e:
    # Muestra un Dataframe con el index y el valor que rompió el Schema
    print(f'Pandera SchemaError: {str(e)}')

Pandera SchemaError: Column 'id' failed element-wise validator number 0: isin(range(0, 10)) failure cases: 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 37, 37


Otra prueba del objeto Check, en este caso quitando el último elemento de la lista de posibles valores para la columna class.

In [183]:
#Validacion las columnas, aceptando nulls, validando valores predefinidos
id_values = list(set(students['id']))
class_values = list(set(students['class']))

schema = pa.DataFrameSchema(
    {
        "id": Column(pa.Int, Check.isin(id_values)),
        "name": Column(str),
        "class": Column(
            str,
            Check.isin(class_values[:-1]),
            unique=False
        ),
        "mark": Column(int),
        "gender": Column(str, nullable=True)
    }
)

try:
    # Quito el último elemento ("Nine") de la lista de posibles "class" para probar la propiedad Check.
    schema.validate(students)
    print("Data Valida")
except pa.errors.SchemaError as e:
    # Muestra un Dataframe con el index y el valor que rompió el Schema
    print(f'Pandera SchemaError: {str(e)}')

Pandera SchemaError: Column 'class' failed element-wise validator number 0: isin(['Six', 'Three', 'Five', 'Seven', 'Eight', 'Fifth', 'Four']) failure cases: Nine, Nine, Nine


Se pueden agregar otro tipo de validaciones sobre los datos, en forma de listado de Checks. 
Por ejemplo:
- Que los valores de la columna id sean mayores que 0, menores que 100, estén en el listado especificado, y que no se permitan duplicados.
- Que los valores de la columna name tengan un mínimo de 3 caracteres y un máximo de 15.
- Que los valores de la columna class no necesariamente sean únicos, y estén dentro de los valores predefinidos.
- Que los valores de la columna mark sean mayores a 10 y menores a 100, que sería la calificación máxima posible.
- Que los valores de la columna sean "male", "female" o "undefined", y permita nulos.

In [184]:
# Más validaciones sobre los datos
schema = pa.DataFrameSchema(
    {
        "id": Column(
            pa.Int, 
            [
                Check.isin(id_values),
                Check.greater_than(0),
                Check.less_than(100)
            ],
            unique = True
        ),
        "name": Column(
            str,
            Check.str_length(min_value=3, max_value=15)
        ),
        "class": Column(
            str,
            Check.isin(class_values),
            unique = False
        ),
        "mark": Column(
            pa.Int,
            [
                Check.less_than(100),
                Check.greater_than(10),
            ]
        ),
        "gender": Column(
            str,
            Check.isin(["male", "female", "undefined"]),
            nullable = True)
    }
)

try:
    schema.validate(students)
    print("Data Valida")
except pa.errors.SchemaError as e:
    # Muestra un Dataframe con el index y el valor que rompió el Schema
    print(f'Pandera SchemaError: {str(e)}')

Data Valida


Se pueden usar funciones lambda dentro de Check para validaciones más personalizadas sobre los datos. 
Se encuentra más desarrollo y detalles en la documentación oficial de pandera.

In [187]:
# Validación utilizando funciones lambda
schema = pa.DataFrameSchema(
    {
        "id": Column(
            pa.Int, 
            Check(lambda s: s > 0),
            unique = True
        ),
        "name": Column(
            str,
            Check(lambda s: len(s) >= 3 and len(s) < 15, element_wise=True),
        ),
        "class": Column(
            str,
            Check(lambda s: len(s) < 6, element_wise=True),
            unique=False
        ),
        "mark": Column(
            pa.Int,
            Check(lambda s: s > 10 and s <= 100, element_wise=True)
        ),
        "gender": Column(
            str,
            Check(lambda s: str(s).startswith("m") or str(s).startswith("f"), element_wise=True),
            nullable=True)
    }
)

try:
    schema.validate(students)
    print("Data Valida")
except pa.errors.SchemaError as e:
    # Muestra un Dataframe con el index y el valor que rompió el Schema
    print(f'Pandera SchemaError: {str(e)}')

Data Valida


In [ ]:
Es posible personalizar los mensajes de error cuando no se cumple alguna de las validaciones.

In [190]:
# Personalizando el mensaje de error
schema = pa.DataFrameSchema(
    {
        "id": Column(
            pa.Int, 
            Check(lambda s: s > 0),
            unique = True
        ),
        "name": Column(
            str,
            Check(lambda s: len(s) >= 3 and len(s) < 15, element_wise=True),
        ),
        "class": Column(
            str,
            Check(lambda s: len(s) < 6, element_wise=True),
            unique=False
        ),
        "mark": Column(
            pa.Int,
            Check(lambda s: s > 20 and s <= 80, element_wise=True, error="La calificacion debe ser mayor a 20 y menor a 80.")
        ),
        "gender": Column(
            str,
            Check(lambda s: str(s).startswith("m") or str(s).startswith("f"), element_wise=True),
            nullable=True)
    }
)

try:
    schema.validate(students)
    print("Data Valida")
except pa.errors.SchemaError as e:
    # Muestra un Dataframe con el index y el valor que rompió el Schema
    print(f'Pandera SchemaError: {str(e)}')

Pandera SchemaError: Column 'mark' failed element-wise validator number 0: <Check <lambda>: La calificacion debe ser mayor a 20 y menor a 80.> failure cases: 85, 85, 89, 94, 88, 88, 88, 88, 18, 88, 81, 86, 88, 90, 96, 88, 89, 95


Hipotesis

In [207]:
from pandera import Hypothesis

# Utilización de Hypothesis - Caso Valido
schema = pa.DataFrameSchema(
    {
        "id": Column(pa.Int),
        "name": Column(str),
        "class": Column(str),
        "mark": Column(
            pa.Int,
            checks=[
                Check.greater_than(0),
                Hypothesis.two_sample_ttest(
                # La idea es que las calificaciones de los estudiantes del grupo SEIS sean mayores a las del grupo UNO.
                sample1="Six",
                relationship="greater_than",
                sample2="One",
                groupby="class",
                alpha=0.01,
                )
            ]
        ),
        "gender": Column(str, nullable=True)
    }
)

try:
    students.loc[37] = [38, 'Pepe', 'One', 25, None]
    students.loc[38] = [39, 'Carl', 'One', 30, None]
    # La hipótesis funciona porque los 2 del grupo ONE tienen calificaciones menores a los del grupo SIX
    schema.validate(students)
    print("Data Valida")
except pa.errors.SchemaError as e:
    # Muestra un Dataframe con el index y el valor que rompió el Schema
    print(f'Pandera SchemaError: {str(e)}')

Data Valida


In [206]:
from pandera import Hypothesis

# Utilización de Hypothesis - Caso Invalido
schema = pa.DataFrameSchema(
    {
        "id": Column(pa.Int),
        "name": Column(str),
        "class": Column(str),
        "mark": Column(
            pa.Int,
            checks=[
                Check.greater_than(0),
                Hypothesis.two_sample_ttest(
                # La idea es que las calificaciones de los estudiantes del grupo NUEVE sean mayores a las del grupo UNO.
                sample1="Nine",
                relationship="greater_than",
                sample2="One",
                groupby="class",
                alpha=0.01,
                )
            ]
        ),
        "gender": Column(str, nullable=True)
    }
)

try:
    students.loc[37] = [38, 'Pepe', 'One', 25, None]
    students.loc[38] = [39, 'Carl', 'One', 30, None]
    # Falla la validación de la hipótesis porque el grupo nueve tiene un alumno con calificación 18.
    schema.validate(students)
    print("Data Valida")
except pa.errors.SchemaError as e:
    # Muestra un Dataframe con el index y el valor que rompió el Schema
    print(f'Pandera SchemaError: {str(e)}')

Pandera SchemaError: Column 'mark' failed series or dataframe validator 1: <Check two_sample_ttest: failed two sample ttest between 'Nine' and 'One'>


Siendo la validación de los datos un aspecto de vital importancia, en particular al manejar grandes datasets, la potencia y facilidad de uso de pandera lo hacen una opción más que interesante para garantizar la sanidad de los datos. 
Me resulta particularmente interesante la validación de hipótesis, siendo de gran utilidad para comprobar fácilmente cualquier hipótesis sobre el dataset.
No obstante, como aclara Niels finalizando su artículo, es el costo computacional de correr las validaciones en tiempo de ejecución con pandera, que puede llegar a ser considerable con datasets grandes.

Referencias:
- Bantilan, Niels. (2020). pandera: Statistical Data Validation of Pandas Dataframes. 116-124. 10.25080/Majora-342d178e-010. Disponible: [pandera_Statistical_Data_Validation_of_Pandas_Dataframes](https://www.researchgate.net/publication/343231859_pandera_Statistical_Data_Validation_of_Pandas_Dataframes)
- pandera documentation. [En línea]. Disponible: [https://pandera.readthedocs.io/en/stable/](https://pandera.readthedocs.io/en/stable/)
- S. Dinc. “Dataframe Validation with Pandera”. Medium. 2023. [En línea]. Disponible: [dataframe-validation-with-pandera](https://medium.com/@seckindinc/dataframe-validation-with-pandera-cacd1a20878f)
- Enthought. Pandera: Statistical Data Validation of Pandas Dataframes |SciPy 2020| Niels Bantilan. (5 de julio de 2020). [Video en línea]. Disponible: [Pandera: Statistical Data Validation of Pandas Dataframes |SciPy 2020| Niels Bantilan](https://www.youtube.com/watch?v=PxTLD-ueNd4)